<a href="https://colab.research.google.com/github/Savoxism/Mathematical-Algorithms/blob/main/gaussian_elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Gaussian elimination offers a systematic approach to solving systems of linear equations by transforming an augmented matrix into row-echelon form, thereby enabling the determination of variables.*

The algorithm comprises several essential steps: **1)** Augmenting the Coefficient Matrix and the Solution Matrix into the Augmented Matrix -> **2)** Transforming the Augmented Matrix into the Row-echelon form (basically zeroes under all the pivots in the main diagonal) -> **3)** Backsubstitution from the bottom up

There are 3 main elementary operations to transform into row-echelon form:

**Row Switching:** bold text Rearrange rows to position the leftmost non-zero entry at the top.

**Row Scaling:** Multiply a row by a non-zero scalar.

**Row Replacement:** Substitute a row with the sum of itself and a multiple of another row.

Though not the most cutting edge method today, Gaussian Elimination is historically significant and provides a solid starting point for understanding the evolution of linear algebra techniques.

In [30]:
import numpy as np
import re

In [31]:
def swap_rows(M, row_index_1, row_index_2):
    """
    Swap rows in the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to perform row swaps on.
    - row_index_1 (int): Index of the first row to be swapped.
    - row_index_2 (int): Index of the second row to be swapped.
    """

    # Copy matrix M so the changes do not affect the original matrix.
    M = M.copy()
    # Swap indexes
    M[[row_index_1, row_index_2]] = M[[row_index_2, row_index_1]]
    return M

In [32]:
def get_index_first_non_zero_value_from_column(M, column, starting_row):
    """
    Retrieve the index of the first non-zero value in a specified column of the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to search for non-zero values.
    - column (int): The index of the column to search.
    - starting_row (int): The starting row index for the search.

    Returns:
    int: The index of the first non-zero value in the specified column, starting from the given row.
                Returns -1 if no non-zero value is found.
    """
    # Get the column array starting from the specified row
    column_array = M[starting_row:,column]
    for i, val in enumerate(column_array):
        # Iterate over every value in the column array.
        # To check for non-zero values, you must always use np.isclose instead of doing "val == 0".
        if not np.isclose(val, 0, atol = 1e-5):
            # If one non zero value is found, then adjust the index to match the correct index in the matrix and return it.
            index = i + starting_row
            return index
    # If no non-zero value is found below it, return -1.
    return -1

def get_index_first_non_zero_value_from_row(M, row, augmented = False):
    """
    Find the index of the first non-zero value in the specified row of the given matrix.

    Parameters:
    - matrix (numpy.array): The input matrix to search for non-zero values.
    - row (int): The index of the row to search.
    - augmented (bool): Pass this True if you are dealing with an augmented matrix,
                        so it will ignore the constant values (the last column in the augmented matrix).

    Returns:
    int: The index of the first non-zero value in the specified row.
                Returns -1 if no non-zero value is found.
    """

    # Create a copy to avoid modifying the original matrix
    M = M.copy()

    # If it is an augmented matrix, then ignore the constant values
    if augmented == True:
        # Isolating the coefficient matrix (removing the constant terms)
        M = M[:,:-1]

    # Get the desired row
    row_array = M[row]
    for i, val in enumerate(row_array):
        # If finds a non zero value, returns the index. Otherwise returns -1.
        if not np.isclose(val, 0, atol = 1e-5):
            return i
    return -1

In [33]:
def augmented_matrix(A, B):
    """
    Combines the coefficient matrix A and the constants vector B into an augmented matrix [A | B].

    Parameters:
    - A (numpy.array): The input square matrix of coefficients.
    - B (numpy.array): The input column matrix of constant terms.

    Returns:
    numpy.array: The augmented matrix [A | B].
    """
    # Reshape B to be a column vector if it's not already
    if B.ndim == 1:
        B = B.reshape(-1, 1)
    return np.hstack((A, B))

In [34]:
def row_echelon_form(A, B):
    """
    Utilizes elementary row operations to transform a given set of matrices,
    which represent the coefficients and constant terms of a linear system, into row echelon form.

    Parameters:
    - A (numpy.array): The input square matrix of coefficients.
    - B (numpy.array): The input column matrix of constant terms

    Returns:
    numpy.array: A new augmented matrix in row echelon form with pivots as 1.
    """

    # Before any computation, check if matrix A (coefficient matrix) has non-zero determinant.
    # It will be used the numpy sub library np.linalg to compute it.

    det_A = np.linalg.det(A)

    # Returns "Singular system" if determinant is zero
    if np.isclose(det_A, 0) == True:
        return 'Singular system'

    # Make copies of the input matrices to avoid modifying the originals
    A = A.copy()
    B = B.copy()

    # Convert matrices to float to prevent integer division
    A = A.astype('float64')
    B = B.astype('float64')

    # Number of rows in the coefficient matrix
    num_rows = len(A)

    # Transform matrices A and B into the augmented matrix M
    M = augmented_matrix(A, B)

    # Iterate over the rows.
    for row in range(num_rows):

        # The first pivot candidate is always in the main diagonal, let's get it.
        pivot_candidate = M[row, row]

        # If pivot_candidate is zero, it cannot be a pivot for this row.
        if np.isclose(pivot_candidate, 0):
            # Get the index of the first non-zero value below the pivot_candidate.
            first_non_zero_value_below_pivot_candidate = get_index_first_non_zero_value_from_column(M, row, row)

            # Swap rows
            M = swap_rows(M, row, first_non_zero_value_below_pivot_candidate)

            # Get the pivot, which is in the main diagonal now
            pivot = M[row, row]

        # If pivot_candidate is already non-zero, then it is the pivot for this row
        else:
            pivot = pivot_candidate

        # Now you are ready to apply the row reduction in every row below the current

        # Divide the current row by the pivot, so the new pivot will be 1.
        M[row] = 1/pivot * M[row]

        # Perform row reduction for rows below the current row
        for j in range(row + 1, num_rows):
            # Get the value in the row that is below the pivot value.
            value_below_pivot = M[j, row]

            # Perform row reduction using the formula:
            M[j] = M[j] - value_below_pivot * M[row]

    return M


In [35]:
def back_substitution(M):
    """
    Perform back substitution on an augmented matrix (with unique solution) in reduced row echelon form to find the solution to the linear system.

    Parameters:
    - M (numpy.array): The augmented matrix in row echelon form with unitary pivots (n x n+1).

    Returns:
    numpy.array: The solution vector of the linear system.
    """

    # Make a copy of the input matrix to avoid modifying the original
    M = M.copy()

    # Get the number of rows (and columns) in the matrix of coefficients
    num_rows = M.shape[0]

    # Iterate from bottom to top
    for row in reversed(range(num_rows)):
        substitution_row = M[row, :]

        # Get the index of the first non-zero element in the substitution row.
        index = row

        # Iterate over the rows above the substitution_row
        for j in range(row):
            # Get the row to be reduced.
            row_to_reduce = M[j, :]

            # Get the value of the element at the found index in the row to reduce
            value = row_to_reduce[index]

            # Perform the back substitution step
            row_to_reduce = row_to_reduce - value * substitution_row

            # Replace the updated row in the matrix
            M[j, :] = row_to_reduce

    # Extract the solution from the last column
    solution = M[:, -1]

    return solution

In [36]:
def gaussian_elimination(A, B):
    """
    Solve a linear system represented by an augmented matrix using the Gaussian elimination method.

    Parameters:
    - A (numpy.array): Square matrix of size n x n representing the coefficients of the linear system
    - B (numpy.array): Column matrix of size 1 x n representing the constant terms.

    Returns:
    numpy.array or str: The solution vector if a unique solution exists, or a string indicating the type of solution.
    """

    # Get the matrix in row echelon form
    row_echelon_M = row_echelon_form(A, B)

    # If the system is non-singular, then perform back substitution to get the result.
    if not isinstance(row_echelon_M, str):
        solution = back_substitution(row_echelon_M)
    else:
        solution = row_echelon_M

    return solution

In [37]:
def string_to_augmented_matrix(equations):
    """
    Converts a string of linear equations into an augmented matrix.

    Parameters:
    - equations (str): A string containing the linear equations.

    Returns:
    tuple: A tuple containing the variables as a string, the coefficient matrix (A), and the constants vector (B).
    """
    # Extract each equation
    eq_list = equations.strip().split('\n')

    # Find all variables in the equations
    variable_set = set()
    for eq in eq_list:
        variable_set.update(re.findall(r'[a-zA-Z]+', eq))

    # Sort variables alphabetically
    variables = sorted(variable_set)

    # Number of equations
    num_eqs = len(eq_list)

    # Number of variables
    num_vars = len(variables)

    # Initialize coefficient matrix A and constants vector B
    A = np.zeros((num_eqs, num_vars))
    B = np.zeros(num_eqs)

    # Fill in the coefficient matrix and constants vector
    for i, eq in enumerate(eq_list):
        # Split the equation into LHS and RHS
        lhs, rhs = eq.split('=')

        # Parse the RHS to get the constant term
        B[i] = float(rhs.strip())

        # Parse the LHS to get the coefficients
        for var in variables:
            # Find coefficients for each variable in the equation
            matches = re.findall(rf'([+-]?\d*\.?\d*)\s*\*?\s*{var}', lhs)
            if matches:
                # If coefficient is not explicitly given, it is 1 or -1
                coeff_str = matches[0].replace(' ', '')
                if coeff_str in ('', '+'):
                    coeff = 1.0
                elif coeff_str == '-':
                    coeff = -1.0
                else:
                    coeff = float(coeff_str)

                # Determine the position of the variable in the matrix
                j = variables.index(var)
                A[i, j] = coeff

    return ' '.join(variables), A, B

In [41]:
#Example Usage
equations = """
6*x + 4*y + 9*w + 2*z = 10
1*x + 2*y + 4*w = -18
5*y - 2*w + 2*z = 8
7*w + 3*z = 9
"""

variables, A, B = string_to_augmented_matrix(equations)

sols = gaussian_elimination(A, B)

if not isinstance(sols, str):
    for variable, solution in zip(variables.split(' '),sols):
        print(f"{variable} = {solution:.4f}")
else:
    print(sols)

w = -4.8134
x = 5.5877
y = -2.1671
z = 14.2312
